# Cleaning data borrowed from a real-estate listing website

In [ ]:
import pandas as pd

In [ ]:
DATA_PATH = "data/"
PATH_TO_APP_MEDIA_ROOT = "../app/media/"

display_max_output = True
if display_max_output:
    pd.options.display.max_rows = 999
    pd.options.display.max_columns = 999

In [ ]:
# This data is from a Canadian real estate listing website: realtor.ca
# I'm using this data for private and non-commercial use
#   as per realtor.ca's Terms of Use Agreement: https://www.realtor.ca/terms-of-use

# To reproduce:
# Copy the network request to https://api2.realtor.ca/Listing.svc/PropertySearch_Post
# Update the `RecordsPerPage` value to be whatever you want.
# Note that this is not an official API.

realtor_api_response = pd.read_json(f"{DATA_PATH}realtor_ca_data.json", lines=True)
realtor_api_response

In [ ]:
results = realtor_api_response["Results"].explode("Results")
properties = pd.json_normalize(results)
properties

In [ ]:
properties.columns

## Separate out Address Text
Before: '100 Regina St. S. Unit# 100|Waterloo, Ontario N2J4P9'  
After: ['100 Regina St. S.', '100', 'Waterloo', 'Ontario', 'N2J4P9', 'Canada']  

In [ ]:
# Separate out street address
split_address_text = (
    properties["Property.Address.AddressText"]
    .str.split("|")
)

properties["Property.Address.StreetAddress"] = (
    split_address_text
    .apply(lambda x: x[0].strip())
)

In [ ]:
# Separate out city
split_city_province = (
    split_address_text.apply(lambda x: x[1])
    .str.split(",")
)

properties["Property.Address.City"] = (
    split_city_province.apply(lambda x: x[0].strip())
)

In [ ]:
# Separate out province
split_province_postal_code = (
    split_city_province.apply(lambda x: x[1].strip())
    .str.split(" ")
)

properties["Property.Address.Province"] = (
    split_province_postal_code.apply(lambda x: x[0].strip())
)

In [ ]:
# Separate out unit number for street addresses with "Unit#"
properties["Property.Address.UnitNumber"] = ""

props_w_unit_num_mask = properties["Property.Address.StreetAddress"].str.contains(" Unit# ")

split_addresses = (
    properties[props_w_unit_num_mask]["Property.Address.StreetAddress"]
    .str.split(" Unit# ")
)

properties.loc[
    props_w_unit_num_mask,
    ["Property.Address.UnitNumber"],
] = split_addresses.apply(lambda x: x[1].strip())

properties.loc[
    props_w_unit_num_mask,
    ["Property.Address.StreetAddress"],
] = split_addresses.apply(lambda x: x[0].strip())

In [ ]:
# Separate out unit number for street addresses with "#__ -street_address"
props_w_unit_num_mask = properties["Property.Address.StreetAddress"].str[0] == "#"

split_addresses = (
    properties[props_w_unit_num_mask]["Property.Address.StreetAddress"]
    .str.split(" -")
)

properties.loc[
    props_w_unit_num_mask,
    ["Property.Address.UnitNumber"],
] = split_addresses.apply(lambda x: x[0][1:].strip())

properties.loc[
    props_w_unit_num_mask,
    ["Property.Address.StreetAddress"],
] = split_addresses.apply(lambda x: x[1].strip())

In [ ]:
# Clean up street address strings
properties["Property.Address.StreetAddress"] = (
    properties["Property.Address.StreetAddress"]
    .str.replace(" +", " ")
    .str.title()
)

In [ ]:
# Add Canada as country. Hard-coded because realtor.ca only operates in Canada.
properties["Property.Address.Country"] = "Canada"

In [ ]:
properties.head()

## Clean up other columns

In [ ]:
# Get the first photo in high res. It's all we need for now. 
properties["Property.PhotoLink"] = (
    properties["Property.Photo"]
    .apply(
        lambda x: x[0]["HighResPath"]
        if isinstance(x, list) and x[0]
        else ""
    )
)

In [ ]:
# Pre-pend realtor website to RelativeDetailsURL
properties["DetailsURL"] = (
    "https://realtor.ca" + properties["RelativeDetailsURL"]
)

In [ ]:
properties["Property.ParkingType"] = (
    properties["Property.Parking"]
    .apply(
        lambda x: x[0]["Name"]
        if isinstance(x, list) and x[0]
        else ""
    )
)

In [ ]:
properties["DateAccessed"] = pd.to_datetime("2021-05-12")

## Export data

In [ ]:
cols_of_interest = [
    "Property.Address.StreetAddress", "Property.Address.UnitNumber",
    "Property.Address.City", "Property.Address.Province", "PostalCode",
    "Property.Address.Country",
    "Property.Address.Latitude", "Property.Address.Longitude",
    
    "MlsNumber", "DateAccessed", "Property.PriceUnformattedValue",
    "DetailsURL", "Building.SizeInterior",
    "Building.Bedrooms", "Building.BathroomTotal",
    "Building.StoriesTotal", "Building.UnitTotal", 
    "Land.SizeTotal", "Land.SizeFrontage",
    "Property.PhotoLink", 
    "Property.Type", "Building.Type", 
    "Property.OwnershipType",
    "Property.ParkingType", "Property.ParkingSpaceTotal",
    "PublicRemarks", 
    # "Id", "Individual", "RelativeDetailsURL",
    # "StatusId", "PhotoChangeDateUTC", "HasNewImageUpdate", "Distance",
    # "RelativeURLEn", "RelativeURLFr",
    # "Building.Ammenities", "Property.Price",
    # "Property.Address.PermitShowAddress", "Property.Address.DisseminationArea",
    # "Property.Photo", "Property.TypeId",
    # "Property.AmmenitiesNearBy", "Property.ConvertedPrice",
    # "Property.OwnershipTypeGroupIds", "Property.ParkingType",
    # "Land.AccessType", "AlternateURL.VideoLink",
    # "AlternateURL.PhotoLink", "AlternateURL.BrochureLink",
    # "Property.Address.AddressText", "Land.LandscapeFeatures",
    # "AlternateURL.MapLink", "Building.SizeExterior",
    # "AlternateURL.DetailsLink", "AlternateURL.SoundLink"
]

properties[cols_of_interest]

In [ ]:
properties[cols_of_interest].to_csv(f"{DATA_PATH}realtor_ca_cleaned_data.csv", index=False)

In [ ]:
properties[cols_of_interest].to_csv(
    f"{PATH_TO_APP_MEDIA_ROOT}realtor_ca_cleaned_data.csv",
    index=False,
)